# Here i want to extract from lichess data features important for future modeling

So here i want to get data in format similar to that: 

columns = ( date, amount of mathes played in that date,ammont of matches played day earllier , amount of matches plyed for previous mounth, ranikng points fluctuation (day vs day before), ranking after last match that day, mean oponent vs my ranking diff that day ) and maybe some other columns that i will consider as important for ranking changes prediction. --> df2

And all of those for different times control (blitz, rapid, bullet)

and other data for winner prediction models, similar columns as without any chnages but some new cols like: time used in game, time for move and maybe some other --> df1

# To Be Continued

In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import pandas as pd
import numpy as np
import arrow
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/MikolajZaborowski/LichessData/main/Chess_data_basic.csv", index_col = "Unnamed: 0")

In [3]:
df.shape

(5364, 15)

In [4]:
len(set(df["black_player_name"]+df["white_player_name"]))

4789

In [5]:
df.head(2)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,e4 e5 Nf3 Nc6 Bc4 Nf6 d4 exd4 e5 d5 exd6 Qxd6 ...,300,70,EvilSaintPL,1753,Leosammie,1735
1,KeBcqcHz,True,standard,blitz,2023-06-28 02:31:45.081000+02:00,2023-06-28 02:33:30.178000+02:00,resign,white,Nf3 Nc6 g3 e5 Bg2 e4 Ng1 d5 d3 f5 dxe4 fxe4 Nc...,300,18,convict-19,1801,EvilSaintPL,1758


In [6]:
df.isna().sum()

id                     0
rated                  0
variant                0
speed                  0
createdAt              0
lastMoveAt             0
status                 0
winner               250
moves                  1
clock                  0
movesperplayer         0
white_player_name      0
white_ranking          0
black_player_name      0
black_ranking          0
dtype: int64

In [7]:
def convert_date_with_offset(date_str):
    return arrow.get(date_str).datetime
df['createdAt'] = df['createdAt'].apply(convert_date_with_offset)
df['lastMoveAt'] = df['lastMoveAt'].apply(convert_date_with_offset)
df["Gametime"]= df["lastMoveAt"]-df["createdAt"]
df['Gametime'] = pd.to_timedelta(df['Gametime'])
df['Gametime'] = df['Gametime'].dt.total_seconds().apply(lambda x: round(x, 1))
df['timepermove']=(df['Gametime']/df["movesperplayer"]).apply(lambda x: round(x, 1))

next think to do is taking insight from moves column, there is a lot of possibility but i will concentrate on:
1) cheking if side castle and if long or short castle
2) taking number of moves before castle ( if no castle - values set to number of moves +1 )
3) groping by opening (for this project i will take in count only first 6 moves - 3 moves per side )
4) number of checks in first 10 moves ( per side )

In [8]:
df.columns

Index(['id', 'rated', 'variant', 'speed', 'createdAt', 'lastMoveAt', 'status',
       'winner', 'moves', 'clock', 'movesperplayer', 'white_player_name',
       'white_ranking', 'black_player_name', 'black_ranking', 'Gametime',
       'timepermove'],
      dtype='object')

In [9]:
# Only timepermove colum had na values (just one), expect winner, where na means 
df.dropna(subset=['timepermove'],inplace=True)

In [10]:
df.isna().sum()

id                     0
rated                  0
variant                0
speed                  0
createdAt              0
lastMoveAt             0
status                 0
winner               250
moves                  0
clock                  0
movesperplayer         0
white_player_name      0
white_ranking          0
black_player_name      0
black_ranking          0
Gametime               0
timepermove            0
dtype: int64

# Checking for side castle 

kingside castle ( short ) is O-O in moves notation and queenside castle ( long ) is O-O-O

In [11]:
pd.options.mode.chained_assignment = None

In [12]:
df["moves"] = df["moves"].str.split(" ")
df["white_moves"]=df["moves"].apply(lambda x: x[::2])
df["black_moves"]=df["moves"].apply(lambda x: x[1::2])

In [13]:
df["white_kingcastle"] = df["white_moves"].apply(lambda x: 1 if "O-O" in x else 0)
df["white_queencastle"] = df["white_moves"].apply(lambda x: 1 if "O-O-O" in x else 0)
df["black_kingcastle"] = df["black_moves"].apply(lambda x: 1 if "O-O" in x else 0)
df["black_queencastle"] = df["black_moves"].apply(lambda x: 1 if "O-O-O" in x else 0)

# num of moves before castle 

In [14]:
def find_index_of_first_occurrence(lst):
    elements = ["O-O", "O-O-O"]
    for element in elements:
        if element in lst:
            return lst.index(element)
    return len(lst) + 1
df["White_moves_before_castle"] = df["white_moves"].apply(find_index_of_first_occurrence)
df["Black_moves_before_castle"] = df["black_moves"].apply(find_index_of_first_occurrence)

In [15]:
df.to_csv("to_visual.csv")

In [16]:
df.to_json("to_visual1.json")

Grouping by opening

In [17]:
df["open6"] = df["moves"].apply(lambda x: x[:4])
df["open6"] = df["open6"].apply(lambda x: " ".join(x))

In [18]:
white_opens = df[df['white_player_name'] == 'EvilSaintPL']["open6"].value_counts().head(20)
black_opens = df[df['black_player_name'] == 'EvilSaintPL']["open6"].value_counts().head(20)

In [19]:
result = list()
def openings2( df ):
    for index, row in df.iterrows():
        if df["white_player_name"][index]=="EvilSaintPL" and df["open6"][index] in white_opens.index:
            result.append("White " + df["open6"][index])
        elif  df["white_player_name"][index]!="EvilSaintPL" and df["open6"][index] in black_opens.index:
            result.append("Black " + df["open6"][index]) 
        else:
            result.append("no opens")
    df["opens1"]=result
    return df

In [20]:
rankresult = list()
def rank( df ):
    for index, row in df.iterrows():
        if df["white_player_name"][index]=="EvilSaintPL":
            rankresult.append(df["white_ranking"][index])
        elif df["black_player_name"][index]=="EvilSaintPL":
            rankresult.append(df["black_ranking"][index])
        else:
            print(index)
    df["my_rank"]=rankresult
    return df

In [21]:
# colorres = list()
# def my_col( df ):
    

In [22]:
df = rank(df)

In [23]:
df = openings2( df )

In [24]:
df.head(1)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6,my_rank,opens1
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,White e4 e5 Nf3 Nc6


In [25]:
temp = pd.get_dummies(df["opens1"])
df = pd.concat([df,temp], axis=1)

In [26]:
df.drop(labels = ["opens1"], axis=1, inplace=True)

In [27]:
df.head(2)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6,my_rank,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,KeBcqcHz,True,standard,blitz,2023-06-28 02:31:45.081000+02:00,2023-06-28 02:33:30.178000+02:00,resign,white,"[Nf3, Nc6, g3, e5, Bg2, e4, Ng1, d5, d3, f5, d...",300,18,convict-19,1801,EvilSaintPL,1758,105.1,5.8,"[Nf3, g3, Bg2, Ng1, d3, dxe4, Nc3, Bg5, Bxf6, ...","[Nc6, e5, e4, d5, f5, fxe4, Nf6, Be7, Bxf6, Qx...",1,0,0,1,17,14,Nf3 Nc6 g3 e5,1758,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [28]:
df.columns

Index(['id', 'rated', 'variant', 'speed', 'createdAt', 'lastMoveAt', 'status',
       'winner', 'moves', 'clock', 'movesperplayer', 'white_player_name',
       'white_ranking', 'black_player_name', 'black_ranking', 'Gametime',
       'timepermove', 'white_moves', 'black_moves', 'white_kingcastle',
       'white_queencastle', 'black_kingcastle', 'black_queencastle',
       'White_moves_before_castle', 'Black_moves_before_castle', 'open6',
       'my_rank', 'Black c4 e5 Nc3 Nf6', 'Black d4 Nf6 Bf4 e6',
       'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6', 'Black d4 d5 Bf4 Nf6',
       'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6', 'Black e4 e5 Bc4 Nf6',
       'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6', 'Black e4 e5 Nf3 Nf6',
       'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5', 'Black e4 e5 f4 exf4',
       'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5', 'Black e4 e6 Nf3 d5',
       'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5', 'Black g3 e5 Bg2 Nf6',
       'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6'

In [29]:
df.drop("id",axis=1,inplace=True)

In [30]:
df = df[df["rated"]==True]

In [31]:
df.head(1)

,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6,my_rank,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens
0,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [32]:
df.winner.value_counts()

winner
white    2406
black    2286
Name: count, dtype: int64

In [33]:
df.shape

(4912, 67)

In [34]:
df = df[df["white_player_name"]!="YoungCapitan"]
df = df[df["black_player_name"]!="YoungCapitan"]

In [35]:
df.drop("variant",axis=1,inplace=True)

In [36]:
df.speed.value_counts()

speed
blitz        3417
bullet        975
rapid         335
classical     153
Name: count, dtype: int64

In [37]:
df.drop("createdAt",axis=1,inplace=True)

In [38]:
df.drop("status",axis=1,)

,rated,speed,lastMoveAt,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6,my_rank,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens
0,True,blitz,2023-06-28 02:44:00.722000+02:00,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,True,blitz,2023-06-28 02:33:30.178000+02:00,white,"[Nf3, Nc6, g3, e5, Bg2, e4, Ng1, d5, d3, f5, d...",300,18,convict-19,1801,EvilSaintPL,1758,105.1,5.8,"[Nf3, g3, Bg2, Ng1, d3, dxe4, Nc3, Bg5, Bxf6, ...","[Nc6, e5, e4, d5, f5, fxe4, Nf6, Be7, Bxf6, Qx...",1,0,0,1,17,14,Nf3 Nc6 g3 e5,1758,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,True,blitz,2023-06-26 00:07:33.812000+02:00,white,"[e4, e5, Nc3, Nf6, Nf3, Nc6, d4, exd4, Nxd4, N...",300,15,Gabaz,1790,EvilSaintPL,1764,173.9,11.6,"[e4, Nc3, Nf3, d4, Nxd4, Qxd4, Bg5, Bh4, Qe3, ...","[e5, Nf6, Nc6, exd4, Nxd4, Qe7, h6, c5, d5, Bd...",0,1,0,0,12,16,e4 e5 Nc3 Nf6,1764,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,blitz,2023-06-26 00:04:36.328000+02:00,black,"[e4, g6, d4, Bg7, Nf3, d6, Bf4, Bd7, Bd3, a6, ...",300,12,EvilSaintPL,1771,Harrybrave,1738,47.0,3.9,"[e4, d4, Nf3, Bf4, Bd3, O-O, c3, e5, Qd2, Qxf4...","[g6, Bg7, d6, Bd7, a6, b5, Nf6, Nh5, Nxf4, O-O...",1,0,1,0,5,9,e4 g6 d4 Bg7,1771,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
4,True,blitz,2023-06-26 00:03:44.461000+02:00,white,"[e4, e5, Nf3, Nc6, d4, exd4, Nxd4, Bc5, Nxc6, ...",300,19,Hakim040,1760,EvilSaintPL,1778,180.4,9.5,"[e4, Nf3, d4, Nxd4, Nxc6, Bc4, O-O, b3, exd5, ...","[e5, Nc6, exd4, Bc5, bxc6, Ne7, O-O, d5, cxd5,...",1,0,1,0,6,6,e4 e5 Nf3 Nc6,1778,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [39]:
df.head(1)

,rated,speed,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6,my_rank,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens
0,True,blitz,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [40]:
df["winner"][df["status"]=="resign"].value_counts()

winner
white    1322
black    1242
Name: count, dtype: int64

In [41]:
df = df.fillna("draw")

In [43]:
def i_won_fe(df):
    EvilSaintPL_won = []
    for index, row in df.iterrows():
        if df["winner"][index] == 'draw':
            EvilSaintPL_won.append(0)
        elif df["white_player_name"][index]=="EvilSaintPL" and df["winner"][index]=='white':
            EvilSaintPL_won.append(1)
        elif  df["black_player_name"][index]=="EvilSaintPL" and df["winner"][index]=='black':
            EvilSaintPL_won.append(1)
        else:
            EvilSaintPL_won.append(-1)
    df['i_won'] = EvilSaintPL_won
    return df

In [44]:
df = i_won_fe(df)

In [46]:
df.head(1)

,rated,speed,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6,my_rank,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens,i_won
0,True,blitz,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,1


In [47]:
df.columns

Index(['rated', 'speed', 'lastMoveAt', 'status', 'winner', 'moves', 'clock',
       'movesperplayer', 'white_player_name', 'white_ranking',
       'black_player_name', 'black_ranking', 'Gametime', 'timepermove',
       'white_moves', 'black_moves', 'white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle',
       'Black_moves_before_castle', 'open6', 'my_rank', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White

In [48]:
def enemy_rank(df):
    enemy_rank_list = []
    for index, row in df.iterrows():
        if df['black_ranking'][index] == df['white_ranking'][index]:
            enemy_rank_list.append(df['my_rank'][index])
        elif df['my_rank'][index] == df['black_ranking'][index]:
            enemy_rank_list.append(df['white_ranking'][index])
        else:
            enemy_rank_list.append(df['black_ranking'][index])
    df['enemy_rank'] = enemy_rank_list
    return df

In [49]:
df = enemy_rank(df)

In [50]:
df['rank_difference'] = df['my_rank'] - df['enemy_rank']

In [51]:
df['is_higher'] = (df['my_rank'] > df['enemy_rank'])

In [53]:
df.head(2)

,rated,speed,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6,my_rank,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens,i_won,enemy_rank,rank_difference,is_higher
0,True,blitz,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,1,1735,18,True
1,True,blitz,2023-06-28 02:33:30.178000+02:00,resign,white,"[Nf3, Nc6, g3, e5, Bg2, e4, Ng1, d5, d3, f5, d...",300,18,convict-19,1801,EvilSaintPL,1758,105.1,5.8,"[Nf3, g3, Bg2, Ng1, d3, dxe4, Nc3, Bg5, Bxf6, ...","[Nc6, e5, e4, d5, f5, fxe4, Nf6, Be7, Bxf6, Qx...",1,0,0,1,17,14,Nf3 Nc6 g3 e5,1758,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,-1,1801,-43,False


In [58]:
df.i_won.unique()

array([ 1, -1,  0], dtype=int64)

In [62]:
game_results = pd.get_dummies(df["i_won"])
game_results.columns = ["defeat", ]

In [63]:
game_results

,-1,0,1
0,False,False,True
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
5674,False,False,True
5675,True,False,False
5676,False,False,True
5677,True,False,False


In [ ]:
final_result=df[]

,speed,clock
0,blitz,300
1,blitz,300
2,blitz,300
3,blitz,300
4,blitz,300
...,...,...
5674,blitz,180
5675,blitz,180
5676,blitz,180
5677,blitz,180
